In [11]:
from IPython.core.display import HTML
display(HTML('<style>.container { width:95% !important; } </style>'))

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

In [13]:
from bandit_learn import *

In [14]:
n_sim = 1
noise_std = 0.1
n_arms=10
n_features=10
confidence_scaling_factor = noise_std

SEED = 42
np.random.seed(SEED)

In [15]:
fl = FeatureLoader(
    datasetDir="../data/TSLA_2020_2022/processed_data/",
    featuresDir="../data/TSLA_2020_2022/features/",
    n_arms=10,
    n_features=10,
)
fl.loadFeatures()
print(fl.T)
T = fl.T

220915


In [16]:
stockLoader = StockLoader("TSLA")

In [17]:
fl.features_df.columns

Index(['Unnamed: 0', 'date', 'features', 'inference', 'confidence'], dtype='object')

In [18]:
b = ContextualBandit(
    T=fl.T, n_arms=fl.n_arms, n_features=fl.n_features, feature_loader=fl, stock_loader=stockLoader, seed=SEED
)

In [19]:
regrets = np.empty((1, T))

model = LinUCB(
    b,
    reg_factor=1.0,
    delta=0.1,
    confidence_scaling_factor=confidence_scaling_factor,
)
model.run()
regrets = np.cumsum(model.regrets)

  1%|          | 1800/220915 [6:16:22<829:40:04, 13.63s/it, total regret=17.7, % optimal arm=9.88%] 

In [ ]:
fig, ax = plt.subplots(figsize=(11, 4), nrows=1, ncols=1)

t = np.arange(T)

mean_regrets = np.mean(regrets, axis=0)
std_regrets = np.std(regrets, axis=0) / np.sqrt(regrets.shape[0])
ax.plot(mean_regrets)
ax.fill_between(t, mean_regrets - 2*std_regrets, mean_regrets + 2*std_regrets, alpha=0.15)
ax.set_title('Cumulative regret')

plt.tight_layout()
plt.show()

fig.savefig('figures/linucb_linear.pdf')